# [第7章: データベース](http://www.cl.ecei.tohoku.ac.jp/nlp100/#ch7)
[artist.json.gz](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/artist.json.gz)は，オープンな音楽データベース[MusicBrainz](https://musicbrainz.org/)の中で，アーティストに関するものをJSON形式に変換し，gzip形式で圧縮したファイルである．このファイルには，1アーティストに関する情報が1行にJSON形式で格納されている．JSON形式の概要は以下の通りである．

| フィールド | 型 | 内容 | 例 |
| :--------- | :- | :--- | :- |
| id | ユニーク識別子 | 整数 | 20660 |
| gid | グローバル識別子 | 文字列 | "ecf9f3a3-35e9-4c58-acaa-e707fba45060" |
| name | アーティスト名 | 文字列 | "Oasis" |
| sort_name | アーティスト名（辞書順整列用） | 文字列 | "Oasis" |
| area | 活動場所 | 文字列 | "United Kingdom" |
| aliases | 別名 | 辞書オブジェクトのリスト |  |
| aliases[].name | 別名 | 文字列 | "オアシス" |
| aliases[].sort_name | 別名（整列用） | 文字列 | "オアシス" |
| begin | 活動開始日 | 辞書 |  |
| begin.year | 活動開始年 | 整数 | 1991 |
| begin.month | 活動開始月 | 整数 |  |
| begin.date | 活動開始日 | 整数 |  |
| end | 活動終了日 | 辞書 |  |
| end.year | 活動終了年 | 整数 | 2009 |
| end.month | 活動終了月 | 整数 | 8 |
| end.date | 活動終了日 | 整数 | 28 |
| tags | タグ | 辞書オブジェクトのリスト |  |
| tags[].count | タグ付けされた回数 | 整数 | 1 |
| tags[].value | タグ内容 | 文字列 | "rock" |
| rating | レーティング | 辞書オブジェクト |  |
| rating.count | レーティングの投票数 | 整数 | 13 |
| rating.value | レーティングの値（平均値） | 整数 | 86 |

artist.json.gzのデータをKey-Value-Store (KVS) およびドキュメント志向型データベースに格納・検索することを考える．KVSとしては，[LevelDB](http://leveldb.org/)，[Redis](http://redis.io/)，[KyotoCabinet](http://fallabs.com/kyotocabinet/)等を用いよ．ドキュメント志向型データベースとして，[MongoDB](http://www.mongodb.org/)を採用したが，[CouchDB](http://couchdb.apache.org/)や[RethinkDB](http://rethinkdb.com/)等を用いてもよい．

In [1]:
!ls ../Corpus/artist.json.gz

../Corpus/artist.json.gz


## 60. KVSの構築
Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．

In [2]:
import gzip
import json
import plyvel

In [52]:
db_60 = plyvel.DB("artist_60", create_if_missing=True)

In [53]:
%%time
with gzip.open("../Corpus/artist.json.gz", mode="rt") as artists_json, db_60.write_batch() as wb_60:
    for artist_json in artists_json:
        artist = json.loads(artist_json)
        if "name" not in artist or "area" not in artist:
            continue
        k = artist["name"]
        v = artist["area"]
        wb_60.put(k.encode("utf-8"), v.encode("utf-8"))

CPU times: user 7.99 s, sys: 68.1 ms, total: 8.06 s
Wall time: 8.09 s


In [54]:
db_60.close()

## 61. KVSの検索
60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ．

In [49]:
db_60 = plyvel.DB("artist_60")

In [50]:
db_60.get(b"Oasis")

b'United Kingdom'

In [51]:
db_60.close()

## 62. KVS内の反復処理
60で構築したデータベースを用い，活動場所が「Japan」となっているアーティスト数を求めよ．

In [55]:
db_60 = plyvel.DB("artist_60")

In [56]:
count = 0
for _, v in db_60:
    if v == b"Japan":
        count += 1
count

22128

In [57]:
db_60.close()

## 63. オブジェクトを値に格納したKVS
KVSを用い，アーティスト名（name）からタグと被タグ数（タグ付けされた回数）のリストを検索するためのデータベースを構築せよ．さらに，ここで構築したデータベースを用い，アーティスト名からタグと被タグ数を検索せよ．

| フィールド | 型 | 内容 | 例 |
| :--------- | :- | :--- | :- |
| name | アーティスト名 | 文字列 | "Oasis" |
| tags | タグ | 辞書オブジェクトのリスト |  |
| tags[].count | タグ付けされた回数 | 整数 | 1 |
| tags[].value | タグ内容 | 文字列 | "rock" |

In [67]:
db_63 = plyvel.DB("artist_63", create_if_missing=True)

In [68]:
%%time
with gzip.open("../Corpus/artist.json.gz", mode="rt") as artists_json, db_63.write_batch() as wb_63:
    i = 0
    for artist_json in artists_json:
        artist = json.loads(artist_json)
        if "name" not in artist or "tags" not in artist:
            continue
        k = artist["name"]
        v = ""
        for tag in artist["tags"]:
            v += "%s=%s, " % (tag["value"], tag["count"])
        wb_63.put(k.encode("utf-8"), v[:-2].encode("utf-8"))

CPU times: user 7.49 s, sys: 52.6 ms, total: 7.54 s
Wall time: 7.6 s


In [69]:
db_63.get(b"Rezerwat")

b'polish rock=1, new wave=1'

In [70]:
db_63.close()

## 64. MongoDBの構築
アーティスト情報（artist.json.gz）をデータベースに登録せよ．さらに，次のフィールドでインデックスを作成せよ: name, aliases.name, tags.value, rating.value

## 65. MongoDBの検索
MongoDBのインタラクティブシェルを用いて，"Queen"というアーティストに関する情報を取得せよ．さらに，これと同様の処理を行うプログラムを実装せよ．

## 66. 検索件数の取得
MongoDBのインタラクティブシェルを用いて，活動場所が「Japan」となっているアーティスト数を求めよ．

## 67. 複数のドキュメントの取得
特定の（指定した）別名を持つアーティストを検索せよ．

## 68. ソート
"dance"というタグを付与されたアーティストの中でレーティングの投票数が多いアーティスト・トップ10を求めよ．

## 69. Webアプリケーションの作成
ユーザから入力された検索条件に合致するアーティストの情報を表示するWebアプリケーションを作成せよ．アーティスト名，アーティストの別名，タグ等で検索条件を指定し，アーティスト情報のリストをレーティングの高い順などで整列して表示せよ．